Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [8]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [9]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [10]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [11]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [12]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 19.238325
Training accuracy: 7.2%
Validation accuracy: 8.9%
Loss at step 100: 2.369190
Training accuracy: 71.4%
Validation accuracy: 70.6%
Loss at step 200: 1.914852
Training accuracy: 74.2%
Validation accuracy: 73.3%
Loss at step 300: 1.664791
Training accuracy: 75.6%
Validation accuracy: 74.0%
Loss at step 400: 1.489835
Training accuracy: 76.5%
Validation accuracy: 74.4%
Loss at step 500: 1.358248
Training accuracy: 77.3%
Validation accuracy: 74.9%
Loss at step 600: 1.255119
Training accuracy: 77.7%
Validation accuracy: 75.4%
Loss at step 700: 1.171747
Training accuracy: 78.5%
Validation accuracy: 75.6%
Loss at step 800: 1.102852
Training accuracy: 78.9%
Validation accuracy: 75.7%
Test accuracy: 82.6%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [18]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [19]:
def run_model(num_steps):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


In [20]:
run_model(10001)

Initialized
Minibatch loss at step 0: 13.318463
Minibatch accuracy: 21.9%
Validation accuracy: 20.1%
Minibatch loss at step 500: 1.941435
Minibatch accuracy: 74.2%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 1.223973
Minibatch accuracy: 76.6%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 1.403852
Minibatch accuracy: 78.1%
Validation accuracy: 77.3%
Minibatch loss at step 2000: 0.970431
Minibatch accuracy: 79.7%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 0.867830
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 1.106812
Minibatch accuracy: 76.6%
Validation accuracy: 78.7%
Minibatch loss at step 3500: 0.981757
Minibatch accuracy: 80.5%
Validation accuracy: 79.6%
Minibatch loss at step 4000: 0.816026
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 4500: 0.731103
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 5000: 0.452485
Minibatch accuracy: 86.7%
Validation accura

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [16]:
batch_size = 128
hidden_nodes = 1024

def model(x, weights1, biases1, weights2, biases2):
    layer_1 = tf.matmul(x, weights1) + biases1
    layer_1_relu = tf.nn.relu(layer_1)
    return tf.matmul(layer_1_relu, weights2) + biases2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = model(tf_train_dataset, weights1, biases1, weights2, biases2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction =  tf.nn.softmax(model(tf_test_dataset, weights1, biases1, weights2, biases2))


In [17]:
run_model(10001)

Initialized
Minibatch loss at step 0: 317.135071
Minibatch accuracy: 7.0%
Validation accuracy: 24.8%
Minibatch loss at step 500: 15.606516
Minibatch accuracy: 81.2%
Validation accuracy: 77.9%
Minibatch loss at step 1000: 7.647616
Minibatch accuracy: 82.0%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 10.219869
Minibatch accuracy: 78.9%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 2.169527
Minibatch accuracy: 85.2%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 1.994952
Minibatch accuracy: 82.0%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 7.727205
Minibatch accuracy: 85.2%
Validation accuracy: 81.6%
Minibatch loss at step 3500: 4.433067
Minibatch accuracy: 78.1%
Validation accuracy: 82.4%
Minibatch loss at step 4000: 6.877839
Minibatch accuracy: 82.8%
Validation accuracy: 83.3%
Minibatch loss at step 4500: 1.292626
Minibatch accuracy: 84.4%
Validation accuracy: 83.2%
Minibatch loss at step 5000: 0.916012
Minibatch accuracy: 89.1%
Validation accu

In [21]:
batch_size = 128
hidden_nodes = 512
layers = 3

def model(x, weights, biases):
    cur_layer = x
    for i in range(layers):
        cur_layer = tf.matmul(cur_layer, weights[i]) + biases[i]
        if (i != layers - 1):
            cur_layer = tf.nn.relu(cur_layer)
    return cur_layer

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = []
  biases = []
  for i in range(layers):
        if (i == 0):
            weights += [tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))]
            biases += [tf.Variable(tf.zeros([hidden_nodes]))]
        elif (i == layers - 1):
            weights += [tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))]
            biases += [tf.Variable(tf.zeros([num_labels]))]
        else:
            weights += [tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes]))]
            biases += [tf.Variable(tf.zeros([hidden_nodes]))]
  
  # Training computation.
  logits = model(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights, biases))
  test_prediction =  tf.nn.softmax(model(tf_test_dataset, weights, biases))

In [22]:
run_model(10001)

Initialized
Minibatch loss at step 0: 3250.076904
Minibatch accuracy: 3.1%
Validation accuracy: 11.8%
Minibatch loss at step 500: 238.552185
Minibatch accuracy: 78.9%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 99.429794
Minibatch accuracy: 85.9%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 197.294815
Minibatch accuracy: 77.3%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 68.886368
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 76.555359
Minibatch accuracy: 83.6%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 88.115211
Minibatch accuracy: 85.9%
Validation accuracy: 82.6%
Minibatch loss at step 3500: 86.300888
Minibatch accuracy: 85.9%
Validation accuracy: 83.2%
Minibatch loss at step 4000: 60.577621
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Minibatch loss at step 4500: 38.518578
Minibatch accuracy: 87.5%
Validation accuracy: 83.8%
Minibatch loss at step 5000: 35.298649
Minibatch accuracy: 83.6%
Vali